In [ ]:
#default_exp funcs

In [ ]:
#export
import os
from torchvision import transforms
from god import LhtFile
import json
import numpy as np
import shutil
import warnings
from tqdm import tqdm

In [ ]:
#export
def lht_listMethodsOfObj(obj):
    '''
    return a dictionary to save attributes and functions of an object.
    '''
    warnings.filterwarnings("error")
    func_list = []
    attr_list = []
    # attribute is a string representing the attribute name
    for attribute in dir(obj):
        # Get attribute value
        try:
            attribute_value = getattr(obj, attribute)
        except:
            continue
        # check that it is callable
        if callable(attribute_value):
            # filter all __prefix methods
            if attribute.startswith('__') == False:
                func_list.append(attribute)
        else:
            attr_list.append(attribute)
    return {'attributes':attr_list, 'functions': func_list}

In [ ]:
#export
def FindFilesByExtension(root_directory, extension):
    ret = []
    for (root, dirs, files) in os.walk(root_directory, topdown=True):
        for f in files:
            if f.endswith(extension):
                fileName = f
                filePath = os.path.join(root,f)
                lhtFile = LhtFile(fileName,filePath)
                ret.append(lhtFile)
    return ret
#

In [ ]:
#hide
root_directory = os.getcwd()
extension = ".ipynb"
ret = FindFilesByExtension(root_directory, extension)
for i in ret:
    print(f"{i.file_name}")

In [ ]:
#export
def Lht_CreateTestDataset(matrix, xSize, ySize, step):
    '''
    generate slide windows
    Parameters:
        matrix: an image array, height X width X channel
        xSize: width of a slide window
        ySize: height of a slide window
        step : length of slide step. 
    Returns:
        rtn: a list of dictionary objects
        {'data':d, 'bbox':bbox, 'category_id': -1}
    '''
    rtn = []
    rowNum = len(matrix)
    colNum = len(matrix[0])
    for i in range(0, colNum, step):
        for j in range(0, rowNum, step):
            bbox = [i,j,xSize,ySize]
            col,row,colSize, rowSize = bbox
            d = matrix[row:row+rowSize, col:col+colSize,:]
            a = {'data':d, 'bbox':bbox, 'category_id': -1}
            rtn.append(a)
    return rtn

In [ ]:
#export
def lht_compose_transforms(size):
    '''
    create torch composed transform object. PILImage object -> resized object -> tensor object
    Parameter:
        size: sequence or int. something like [h,w]
    Return:
        rtn: torchvision.transforms.transforms.Compose object
    '''
    ret = transforms.Compose([transforms.ToPILImage(), transforms.Resize(size),  transforms.ToTensor()])
    return ret

In [ ]:
#hide
ret = lht_compose_transforms([224,224])
type(ret)

torchvision.transforms.transforms.Compose

In [ ]:
#export
def lht_load_json(file_path):
    try:
        with open(file_path, 'r') as f:
            obj = json.load(f)
        return obj
    except FileNotFoundError as err:
        print(err)
        return -1
    

In [ ]:
#hide
ret = lht_load_json("321.json")
ret

[Errno 2] No such file or directory: '321.json'


-1

In [ ]:
#export
def randomSelectSamples(jsonList, num):
    '''
    randomly select 'num' images from each folder.
    Parameters:
        jsonList: a list of json files
        num: the number of you want
    Return:
        ret: a dictionary. 
            - ['images', 'type', 'annotations', 'categories']
    '''
    ret = {}
    for js_id, js in enumerate(jsonList):
        try:
            obj_json = lht_load_json(js.file_path)
            if obj_json == -1:
                raise FileNotFoundError
            n = len(obj_json['images'])

            if n == 0:
                raise ValueError("No value!")
            #
            dirName = os.path.dirname(js.file_path)
            objs = np.random.choice(obj_json['images'], num, replace=False)
            obj_ids = [i['id'] for i in objs]
            for obj in objs:
                fp = dirName + "\\" + obj['file_name']
                obj['file_path'] = fp
            annotation = filter(lambda i: i['image_id'] in obj_ids, obj_json['annotations'])
            annotation = list(annotation)
            #
            d = {'images': objs.tolist(), 'annotations': annotation, 'categories':obj_json['categories']}
            ret[js_id] = d
            #        
        except FileNotFoundError as err:
            continue
        except ValueError as err:
            continue
    return ret
#

In [ ]:
#export
def lht_generate_cocojson(oldjson):
    images = []
    annotations = []
    category = [{'id':0,'category_name': "None"}, {'id':1,'category_name': "animal"}]
    #
    im_idx = 0
    bb_idx = 0
    for i in oldjson.values():
        for im in i['images']:
            im_id = im['id']
            im_ann = filter(lambda m: m['image_id']==im_id, i['annotations'])
            im_ann = list(im_ann)
            #
            im_newID = im_idx
            im['id'] = im_newID
            images.append(im)
            for an in im_ann:
                an['image_id'] = im_newID
                an['id'] = bb_idx
                an['category_id'] = 1
                bb_idx += 1
                annotations.append(an)
            #
            im_idx += 1
    #
    ret = {'images':images, 'annotations':annotations, 'categories': category}
    return ret
#

In [ ]:
#export
def lhtGenerateTestData(source, destination, num):
    '''
    randomly copy n images from source folder to destination.
    Parameters:
        source: source directory path
        destination: destination path
        num: the number of images
    Return:
        ret_json: a json file that record the detail info of images
    '''
    target_dir = destination
    rd = source
    jsonList = FindFilesByExtension(rd,".json")
    ret_json = randomSelectSamples(jsonList,num)
    if os.path.exists(target_dir):
        shutil.rmtree(target_dir)
    os.mkdir(target_dir)
    for v in ret_json.values():
        for v_im in v['images']:
            shutil.copy(v_im['file_path'], target_dir)
            v_im.pop('file_path', None)
    #
    ret_coco = lht_generate_cocojson(ret_json)
    fp = target_dir + "\\" + "test.json"
    with open(fp, 'w') as fin:
        json.dump(ret_coco, fin)
    # 
    return ret_coco

In [ ]:
#export
def searchImageByName(image_name,js_dict):
    '''
    search images by its name, and return its index in a json file.
    Parameters:
        image_name: an image's name
        js_dict: a dictionary loaded from a json file, where save all details
    Return:
        a list where save the search results.
    '''
    images = js_dict['images']
    m = filter(lambda i: i['file_name'] == image_name, images)
    m = list(m)
    return m if len(m) >0 else -1
    

In [ ]:
#export
def Lht_SlideWindow(matrix,xSize, ySize, step):
    rowNum = len(matrix)
    colNum = len(matrix[0])
    rtn = []
    for i in range(0, colNum, step):
        for j in range(0, rowNum, step):
            bbox = [i,j,xSize,ySize]
            rtn.append(bbox)
    return rtn
    pass

def isInclude(bbox1, bbox2):
    '''
    judge whether bbox1 contains bbox2
    '''
    x1,y1,xs1, ys1 = bbox1
    x2,y2,xs2, ys2 = bbox2
    rtn = x1 <= x2 and y1 <= y2 and x1+xs1 >= x2+xs2 and y1+ys1 >= y2+ys2
    return rtn

#
def CreateAnnotation(**kwargs):
    '''
    invoke the following functions:
        Lht_SlideWindow, isInclude
    '''
    annotations = []
    root_directory = kwargs['root_directory']
    image = kwargs['image']
    xSize = kwargs['xSize']
    ySize = kwargs['ySize']
    step  = kwargs['step']
    bboxes = kwargs['bboxes']
    #
    image_id = image['id']
    img_file_name = image['file_name']
    img_path = os.path.join(root_directory, img_file_name)
    img = cv2.imread(img_path)
    #
    def fn_contain_yes(sw):
        rtn = []
        for i in bboxes:
            bbox2 = i['bbox']
            a = isInclude(sw, bbox2)
            rtn.append(a)
        return np.any(rtn)
        
    def fn_contain_no(sw):
        rtn = []
        for i in bboxes:
            bbox2 = i['bbox']
            a = isInclude(sw, bbox2)
            rtn.append(not a)
        return np.all(rtn)
        
    #
    sws = Lht_SlideWindow(img, xSize,ySize, step)
    a = filter(fn_contain_yes, sws)
    bboxes_contain_yes = list(a)
    a = filter(fn_contain_no, sws)
    bboxes_contain_no = list(a)
    idx_no = np.random.choice(len(bboxes_contain_no),len(bboxes_contain_yes), replace=False)
    # category_id: 1 -> animal; 0 -> no animal
    for i in bboxes_contain_yes:
        tmp = {'bbox': i, 'image_id':image_id, 'category_id':1}
        annotations.append(tmp)
    #
    for j in idx_no:
        i = bboxes_contain_no[j]
        tmp = {'bbox': i, 'image_id':image_id, 'category_id':0}
        annotations.append(tmp)
    return annotations
    pass
#
def GenerateTrainDateByJson(**kwargs):
    '''
    generate training dataset by json file
    invoke the function: CreateAnnotation
    parameters:
        root_directory
        json: Lht_File object
        xSize: 
        ySize:
        step: the sliding length of slide window
        desc: simple description used in tqdm
    '''
    annotations = []
    root_directory = kwargs['root_directory']
    obj_json = kwargs['json']
    xSize = kwargs['xSize']
    ySize = kwargs['ySize']
    step  = kwargs['step']
    desc  = '' if kwargs['desc'] is None else kwargs['desc']
    #
    rtn_images = []
    images = obj_json['images']
    old_annotations = obj_json['annotations']
    categories = [{'id':1, 'name':'animal'}, {'id':0, 'name':'others'}]
    for image in tqdm(images, desc=desc):
        # judge whether there exists a image
        img_file_name = image['file_name']
        img_path = os.path.join(root_directory, img_file_name)
        if not os.path.exists(img_path):
            continue
        # save availabe image to a list
        rtn_images.append(image)
        #
        fun = lambda x: x['image_id']== image['id']
        d = filter(fun, old_annotations)
        d = list(d)
        #
        a = {'image':image, 'xSize':xSize, 'ySize':ySize, 'step':step, 'bboxes':d, 'root_directory': root_directory}
        r = CreateAnnotation(**a)
        annotations += r 
    #
    rtn = {'images':rtn_images, 'annotations': annotations, 'categories':categories, 'root':root_directory}
    return rtn
    pass
#
def LhtConstructDSByImageID(image_id, ds):
    annotations = ds['annotations']
    images = ds['images']
    root_directory = ds['root']
    a = filter(lambda x: x['id'] == image_id, images)
    a = list(a)
    image = a[0]
    img_path = os.path.join(root_directory, image['file_name'])
    # read an image, and then convert it to be grayscale
    img = Image.open(img_path)
    #img = ImageOps.grayscale(img)
    img = np.asarray(img)
    #img = np.stack((img,img,img), axis=2)
    #
    ans = filter(lambda x: x['image_id']==image_id, annotations)
    ans = list(ans)
    rtn = []
    for i in ans:
        bbox = np.int32(i['bbox'])
        col,row,colSize, rowSize = bbox
        d = img[row:row+rowSize, col:col+colSize,:]
        f = i['category_id']
        a = {'data':d, 'bbox':bbox, 'category_id': f}
        rtn.append(a)
    #
    return rtn
    pass
#
def LhtGetDataByJson(ds):
    rtn = []   
    images = ds['images']
    for image in tqdm(images,desc=ds['root']):
        image_id = image['id']
        a = LhtConstructDSByImageID(image_id, ds)
        rtn += a
    #
    return rtn
    pass
#

In [ ]:
# hide
from nbdev.export import notebook2script
notebook2script()